In [7]:
import pandas as pd
import numpy as np

# 1. 데이터 로드
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def calculate_prospect_score(df):
    # (1) 공격 기여도: 득점 관련 핵심 능력치 합산
    df['Attacking_Score'] = df[['Finishing', 'ShotPower', 'STRating', 'Positioning']].mean(axis=1)
    
    # (2) 기술 및 기동성: 드리블과 속도
    df['Technique_Score'] = df[['PaceTotal', 'DribblingTotal', 'BallControl', 'Agility']].mean(axis=1)
    
    # (3) 멘탈 및 지능: 반응 속도와 침착함 (성장 가능성 지표)
    df['Mental_IQ'] = df[['Reactions', 'Composure', 'Vision']].mean(axis=1)
    
    # (4) 종합 잠재력 지수 계산 (수식)
    # 나이가 어릴수록(Age-14가 작을수록) 점수가 기하급수적으로 높아짐
    base_skill = (df['Attacking_Score'] * 0.5) + (df['Technique_Score'] * 0.25) + (df['Mental_IQ'] * 0.25)
    df['Final_Prospect_Score'] = base_skill / np.log1p(df['Age'] - 14)
    
    return df

# 2. 지표 계산 적용
test_scored = calculate_prospect_score(test)

# 3. 공격수 포지션 필터링 (ST, LW, RW, CF 등)
fw_positions = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF']
attackers = test_scored[test_scored['Position'].isin(fw_positions)].copy()

# 4. 유망주 기준 적용 (23세 이하) 및 정렬
young_attackers = attackers[attackers['Age'] <= 23]
top_10_prospects = young_attackers.sort_values(by='Final_Prospect_Score', ascending=False).head(10)

# 5. 필요한 컬럼만 정리하여 출력 및 저장
output_cols = [
    'ID', 'Age', 'Position', 'Finishing', 'STRating', 
    'PaceTotal', 'Reactions', 'Final_Prospect_Score'
]
final_result = top_10_prospects[output_cols]

# 화면 출력
print("\n" + "="*90)
print("✅ 분석 완료: 차세대 공격수 유망주 TOP 10 리스트")
print("-" * 90)
print(final_result.to_string(index=False))
print("="*90)

# 6. CSV 파일로 저장
file_name = 'top_10_forward_prospects.csv'
final_result.to_csv(file_name, index=False, encoding='utf-8-sig')

# print(f"\n💾 결과가 '{file_name}' 파일로 성공적으로 저장되었습니다.")


✅ 분석 완료: 차세대 공격수 유망주 TOP 10 리스트
------------------------------------------------------------------------------------------
       ID  Age Position  Finishing  STRating  PaceTotal  Reactions  Final_Prospect_Score
TEST_1066   16       ST     7.1429    5.8904     4.7826     5.1562              5.213243
TEST_1308   17       ST     7.9121    7.1233     8.1159     5.0000              5.018636
TEST_1224   16       ST     5.7143    5.2055     6.9565     3.4375              4.607079
TEST_0763   17       LW     6.0440    6.1644     8.4058     3.1250              4.492126
TEST_1144   17       ST     7.0330    5.8904     5.9420     4.2188              4.200737
TEST_0386   19       ST     8.0220    7.8082     7.8261     6.5625              4.152131
TEST_0907   16       ST     5.3846    4.5205     5.3623     2.3438              4.052525
TEST_0996   17       ST     6.7033    5.7534     5.6522     4.6875              3.996522
TEST_1002   20       ST     8.1319    8.0822     7.6812     7.0312         